In [1]:
!git clone https://github.com/zhixuhao/unet.git
!cp -r unet/* ./
!pip install keras-rectified-adam
import tensorflow as tf

tf.__version__

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.91 MiB | 38.81 MiB/s, done.
Resolving deltas: 100% (80/80), done.
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.17.0-cp36-none-any.whl size=14781 sha256=256cd1f1fab127b2841c316d1b38e2f5befeaead2c6dfb6f10a89c18a00ff6a3
  Stored in directory: /root/.cache/pip/wheels/7b/01/27/3a934e1a5644f5b93c720422a6ef97034ea78a21ba71cfb549
Successfully built keras-rectified-adam


'1.15.0'

In [2]:
%%writefile keras_octave_conv.py
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import *

__all__ = ['OctaveConv2D', 'octave_conv_2d']


class OctaveConv2D(Layer):
    """Octave convolutions.
    # Arguments
        octave: The division of the spatial dimensions by a power of 2.
        ratio_out: The ratio of filters for lower spatial resolution.
    # References
        - [Drop an Octave: Reducing Spatial Redundancy in Convolutional Neural Networks with Octave Convolution]
          (https://arxiv.org/pdf/1904.05049.pdf)
    """

    def __init__(self,
                 filters,
                 kernel_size=(3,3),
                 octave=2,
                 ratio_out=0.125,
                 strides=(1, 1),
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=False,
                 use_transpose=False,
                 kernel_initializer='he_normal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(OctaveConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.octave = octave
        self.ratio_out = ratio_out
        self.strides = strides
        self.data_format = data_format
        self.dilation_rate = dilation_rate
        self.use_bias = use_bias
        self.use_transpose = use_transpose
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer
        self.activity_regularizer = activity_regularizer
        self.kernel_constraint = kernel_constraint
        self.bias_constraint = bias_constraint

        self.filters_low = int(filters * self.ratio_out)
        self.filters_high = filters - self.filters_low

        self.conv_high_to_high, self.conv_low_to_high = None, None
        if self.use_transpose:
          if self.filters_high > 0:
              self.conv_high_to_high = self._init_transconv(self.filters_high, name='{}-Trans-Conv2D-HH'.format(self.name))
              self.conv_low_to_high = self._init_transconv(self.filters_high, name='{}-Conv2D-LH'.format(self.name))
          self.conv_low_to_low, self.conv_high_to_low = None, None
          if self.filters_low > 0:
              self.conv_low_to_low = self._init_transconv(self.filters_low, name='{}-Trans-Conv2D-HL'.format(self.name))
              self.conv_high_to_low = self._init_transconv(self.filters_low, name='{}-Trans-Conv2D-LL'.format(self.name))
          self.pooling = AveragePooling2D(
              pool_size=self.octave,
              padding='valid',
              data_format=data_format,
              name='{}-AveragePooling2D'.format(self.name),
          )
          self.up_sampling = UpSampling2D(
              size=self.octave,
              data_format=data_format,
              name='{}-UpSampling2D'.format(self.name)
          )
        else:
          if self.filters_high > 0:
              self.conv_high_to_high = self._init_conv(self.filters_high, name='{}-Conv2D-HH'.format(self.name))
              self.conv_low_to_high = self._init_conv(self.filters_high, name='{}-Conv2D-LH'.format(self.name))
          self.conv_low_to_low, self.conv_high_to_low = None, None
          if self.filters_low > 0:
              self.conv_low_to_low = self._init_conv(self.filters_low, name='{}-Conv2D-HL'.format(self.name))
              self.conv_high_to_low = self._init_conv(self.filters_low, name='{}-Conv2D-LL'.format(self.name))
          self.pooling = AveragePooling2D(
              pool_size=self.octave,
              padding='valid',
              data_format=data_format,
              name='{}-AveragePooling2D'.format(self.name),
          )
          self.up_sampling = UpSampling2D(
              size=self.octave,
              data_format=data_format,
              name='{}-UpSampling2D'.format(self.name)
          )
    def _init_transconv(self, filters, name):
        return Conv2DTranspose(
            filters=filters,
            kernel_size=self.kernel_size,
            strides=self.strides,
            padding='same',
            data_format=self.data_format,
            dilation_rate=self.dilation_rate,
            use_bias=self.use_bias,
            kernel_initializer=self.kernel_initializer,
            bias_initializer=self.bias_initializer,
            kernel_regularizer=self.kernel_regularizer,
            bias_regularizer=self.bias_regularizer,
            activity_regularizer=self.activity_regularizer,
            kernel_constraint=self.kernel_constraint,
            bias_constraint=self.bias_constraint,
            name=name,
        )

    def _init_conv(self, filters, name):
        return Conv2D(
            filters=filters,
            kernel_size=self.kernel_size,
            strides=self.strides,
            padding='same',
            data_format=self.data_format,
            dilation_rate=self.dilation_rate,
            use_bias=self.use_bias,
            kernel_initializer=self.kernel_initializer,
            bias_initializer=self.bias_initializer,
            kernel_regularizer=self.kernel_regularizer,
            bias_regularizer=self.bias_regularizer,
            activity_regularizer=self.activity_regularizer,
            kernel_constraint=self.kernel_constraint,
            bias_constraint=self.bias_constraint,
            name=name,
        )

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape_high, input_shape_low = input_shape
        else:
            input_shape_high, input_shape_low = input_shape, None
        if self.data_format == 'channels_first':
            channel_axis, rows_axis, cols_axis = 1, 2, 3
        else:
            rows_axis, cols_axis, channel_axis = 1, 2, 3
        if input_shape_high[channel_axis] is None:
            raise ValueError('The channel dimension of the higher spatial inputs '
                             'should be defined. Found `None`.')
        if input_shape_low is not None and input_shape_low[channel_axis] is None:
            raise ValueError('The channel dimension of the lower spatial inputs '
                             'should be defined. Found `None`.')
        if input_shape_high[rows_axis] is not None and input_shape_high[rows_axis] % self.octave != 0 or \
           input_shape_high[cols_axis] is not None and input_shape_high[cols_axis] % self.octave != 0:
            raise ValueError('The rows and columns of the higher spatial inputs should be divisible by the octave. '
                             'Found {} and {}.'.format(input_shape_high, self.octave))
        if input_shape_low is None:
            self.conv_low_to_high, self.conv_low_to_low = None, None

        if self.conv_high_to_high is not None:
            with K.name_scope(self.conv_high_to_high.name):
                self.conv_high_to_high.build(input_shape_high)
        if self.conv_low_to_high is not None:
            with K.name_scope(self.conv_low_to_high.name):
                self.conv_low_to_high.build(input_shape_low)
        if self.conv_high_to_low is not None:
            with K.name_scope(self.conv_high_to_low.name):
                self.conv_high_to_low.build(input_shape_high)
        if self.conv_low_to_low is not None:
            with K.name_scope(self.conv_low_to_low.name):
                self.conv_low_to_low.build(input_shape_low)
        super(OctaveConv2D, self).build(input_shape)

    @property
    def trainable_weights(self):
        weights = []
        if self.conv_high_to_high is not None:
            weights += self.conv_high_to_high.trainable_weights
        if self.conv_low_to_high is not None:
            weights += self.conv_low_to_high.trainable_weights
        if self.conv_high_to_low is not None:
            weights += self.conv_high_to_low.trainable_weights
        if self.conv_low_to_low is not None:
            weights += self.conv_low_to_low.trainable_weights
        return weights

    @property
    def non_trainable_weights(self):
        weights = []
        if self.conv_high_to_high is not None:
            weights += self.conv_high_to_high.non_trainable_weights
        if self.conv_low_to_high is not None:
            weights += self.conv_low_to_high.non_trainable_weights
        if self.conv_high_to_low is not None:
            weights += self.conv_high_to_low.non_trainable_weights
        if self.conv_low_to_low is not None:
            weights += self.conv_low_to_low.non_trainable_weights
        return weights

    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            input_shape_high, input_shape_low = input_shape
        else:
            input_shape_high, input_shape_low = input_shape, None

        output_shape_high = None
        if self.filters_high > 0:
            output_shape_high = self.conv_high_to_high.compute_output_shape(input_shape_high)
        output_shape_low = None
        if self.filters_low > 0:
            output_shape_low = self.conv_high_to_low.compute_output_shape(
                self.pooling.compute_output_shape(input_shape_high),
            )

        if self.filters_low == 0:
            return output_shape_high
        if self.filters_high == 0:
            return output_shape_low
        return [output_shape_high, output_shape_low]

    def call(self, inputs, **kwargs):
        if isinstance(inputs, list):
            inputs_high, inputs_low = inputs
        else:
            inputs_high, inputs_low = inputs, None

        outputs_high_to_high, outputs_low_to_high = 0.0, 0.0
        if self.use_transpose:
          if self.conv_high_to_high is not None:
              outputs_high_to_high = self.conv_high_to_high(inputs_high)
          if self.conv_low_to_high is not None:
              outputs_low_to_high = self.up_sampling(self.conv_low_to_high(inputs_low))
          outputs_high = outputs_high_to_high + outputs_low_to_high

          outputs_low_to_low, outputs_high_to_low = 0.0, 0.0
          if self.conv_low_to_low is not None:
              outputs_low_to_low = self.conv_low_to_low(inputs_low)
          if self.conv_high_to_low is not None:
              outputs_high_to_low = self.pooling(self.conv_high_to_low(inputs_high))
          outputs_low = outputs_low_to_low + outputs_high_to_low

          if self.filters_low == 0:
              return outputs_high
          if self.filters_high == 0:
              return outputs_low
        else:
          if self.conv_high_to_high is not None:
              outputs_high_to_high = self.conv_high_to_high(inputs_high)
          if self.conv_low_to_high is not None:
              outputs_low_to_high = self.up_sampling(self.conv_low_to_high(inputs_low))
          outputs_high = outputs_high_to_high + outputs_low_to_high

          outputs_low_to_low, outputs_high_to_low = 0.0, 0.0
          if self.conv_low_to_low is not None:
              outputs_low_to_low = self.conv_low_to_low(inputs_low)
          if self.conv_high_to_low is not None:
              outputs_high_to_low = self.conv_high_to_low(self.pooling(inputs_high))
          outputs_low = outputs_low_to_low + outputs_high_to_low

          if self.filters_low == 0:
              return outputs_high
          if self.filters_high == 0:
              return outputs_low
        return [outputs_high, outputs_low]

    def get_config(self):
        config = {
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'octave': self.octave,
            'ratio_out': self.ratio_out,
            'strides': self.strides,
            'data_format': self.data_format,
            'dilation_rate': self.dilation_rate,
            'use_bias': self.use_bias,
            'kernel_initializer': self.kernel_initializer,
            'bias_initializer': self.bias_initializer,
            'kernel_regularizer': self.kernel_regularizer,
            'bias_regularizer': self.bias_regularizer,
            'activity_regularizer': self.activity_regularizer,
            'kernel_constraint': self.kernel_constraint,
            'bias_constraint': self.bias_constraint
        }
        base_config = super(OctaveConv2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Writing keras_octave_conv.py


In [3]:
%%writefile model.py
# model code all in this cell

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import numpy as np
import tensorflow as tf
from sklearn.utils import class_weight

from keras_radam.training import RAdamOptimizer
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras_octave_conv import OctaveConv2D
      
      
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    # downsampling for lower
    low = layers.AveragePooling2D(2)(inputs)
    high1, low1 = OctaveConv2D(64)([inputs,low])
    high1 = layers.BatchNormalization()(high1)
    high1 = layers.Activation("relu")(high1)
    low1 = layers.BatchNormalization()(low1)
    low1 = layers.Activation("relu")(low1)
    high1, low1 = OctaveConv2D(64)([high1, low1])
    high1 = layers.BatchNormalization()(high1)
    high1 = layers.Activation("relu")(high1)
    low1 = layers.BatchNormalization()(low1)
    low1 = layers.Activation("relu")(low1)
    pool1high = layers.MaxPooling2D(2)(high1)
    pool1low = layers.MaxPooling2D(2)(low1)
    
    high2, low2 = OctaveConv2D(128)([pool1high,pool1low])
    high2 = layers.BatchNormalization()(high2)
    high2 = layers.Activation("relu")(high2)
    low2 = layers.BatchNormalization()(low2)
    low2 = layers.Activation("relu")(low2)
    high2, low2 = OctaveConv2D(128)([high2, low2])
    high2 = layers.BatchNormalization()(high2)
    high2 = layers.Activation("relu")(high2)
    low2 = layers.BatchNormalization()(low2)
    low2 = layers.Activation("relu")(low2)
    pool2high = layers.MaxPooling2D(2)(high2)
    pool2low = layers.MaxPooling2D(2)(low2)
    
    high3, low3 = OctaveConv2D(256)([pool2high,pool2low])
    high3 = layers.BatchNormalization()(high3)
    high3 = layers.Activation("relu")(high3)
    low3 = layers.BatchNormalization()(low3)
    low3 = layers.Activation("relu")(low3)
    high3, low3 = OctaveConv2D(256)([high3, low3])
    high3 = layers.BatchNormalization()(high3)
    high3 = layers.Activation("relu")(high3)
    low3 = layers.BatchNormalization()(low3)
    low3 = layers.Activation("relu")(low3)
    pool3high = layers.MaxPooling2D(2)(high3)
    pool3low = layers.MaxPooling2D(2)(low3)
    
    high4, low4 = OctaveConv2D(512)([pool3high,pool3low])
    high4 = layers.BatchNormalization()(high4)
    high4 = layers.Activation("relu")(high4)
    low4 = layers.BatchNormalization()(low4)
    low4 = layers.Activation("relu")(low4)
    high4, low4 = OctaveConv2D(512)([high4, low4])
    high4 = layers.BatchNormalization()(high4)
    high4 = layers.Activation("relu")(high4)
    low4 = layers.BatchNormalization()(low4)
    low4 = layers.Activation("relu")(low4)
    pool4high = layers.MaxPooling2D(2)(high4)
    pool4low = layers.MaxPooling2D(2)(low4)

    high5, low5 = OctaveConv2D(1024)([pool4high, pool4low])
    high5 = layers.BatchNormalization()(high5)
    high5 = layers.Activation("relu")(high5)
    low5 = layers.BatchNormalization()(low5)
    low5 = layers.Activation("relu")(low5)
    high5 = Dropout(0.4)(high5)
    low5 = Dropout(0.4)(low5)
    high5, low5 = OctaveConv2D(1024)([high5, low5])
    high5 = layers.BatchNormalization()(high5)
    high5 = layers.Activation("relu")(high5)
    low5 = layers.BatchNormalization()(low5)
    low5 = layers.Activation("relu")(low5)
    high5 = Dropout(0.4)(high5)
    low5 = Dropout(0.4)(low5)
    
    uphigh6, uplow6 = OctaveConv2D(512, use_transpose=True, strides=(2,2))([high5,low5])
    uphigh6 = layers.BatchNormalization()(uphigh6)
    uphigh6 = layers.Activation("relu")(uphigh6)
    uplow6 = layers.BatchNormalization()(uplow6)
    uplow6 = layers.Activation("relu")(uplow6)
    merge6high = concatenate([high4,uphigh6], axis = 3)
    merge6low = concatenate([low4,uplow6], axis = 3)
    high6, low6 = OctaveConv2D(512)([merge6high,merge6low])
    high6 = layers.BatchNormalization()(high6)
    high6 = layers.Activation("relu")(high6)
    low6 = layers.BatchNormalization()(low6)
    low6 = layers.Activation("relu")(low6)
    high6, low6 = OctaveConv2D(512)([high6, low6])
    high6 = layers.BatchNormalization()(high6)
    high6 = layers.Activation("relu")(high6)
    low6 = layers.BatchNormalization()(low6)
    low6 = layers.Activation("relu")(low6)


    uphigh7, uplow7 = OctaveConv2D(256, use_transpose=True, strides=(2,2))([high6, low6])
    uphigh7 = layers.BatchNormalization()(uphigh7)
    uphigh7 = layers.Activation("relu")(uphigh7)
    uplow7 = layers.BatchNormalization()(uplow7)
    uplow7 = layers.Activation("relu")(uplow7)
    merge7high = concatenate([high3,uphigh7], axis = 3)
    merge7low = concatenate([low3,uplow7], axis = 3)
    high7, low7 = OctaveConv2D(256)([merge7high, merge7low])
    high7 = layers.BatchNormalization()(high7)
    high7 = layers.Activation("relu")(high7)
    low7 = layers.BatchNormalization()(low7)
    low7 = layers.Activation("relu")(low7)
    high7, low7 = OctaveConv2D(256)([high7, low7])
    high7 = layers.BatchNormalization()(high7)
    high7 = layers.Activation("relu")(high7)
    low7 = layers.BatchNormalization()(low7)
    low7 = layers.Activation("relu")(low7)

    uphigh8, uplow8 = OctaveConv2D(128, use_transpose=True, strides=(2,2))([high7, low7])
    uphigh8 = layers.BatchNormalization()(uphigh8)
    uphigh8 = layers.Activation("relu")(uphigh8)
    uplow8 = layers.BatchNormalization()(uplow8)
    uplow8 = layers.Activation("relu")(uplow8)
    merge8high = concatenate([high2,uphigh8], axis = 3)
    merge8low = concatenate([low2,uplow8], axis = 3)
    high8, low8 = OctaveConv2D(128)([merge8high, merge8low])
    high8 = layers.BatchNormalization()(high8)
    high8 = layers.Activation("relu")(high8)
    low8 = layers.BatchNormalization()(low8)
    low8 = layers.Activation("relu")(low8)
    high8, low8 = OctaveConv2D(128)([high8, low8])
    high8 = layers.BatchNormalization()(high8)
    high8 = layers.Activation("relu")(high8)
    low8 = layers.BatchNormalization()(low8)
    low8 = layers.Activation("relu")(low8)

    uphigh9, uplow9 = OctaveConv2D(64, use_transpose=True, strides=(2,2))([high8, low8])
    uphigh9 = layers.BatchNormalization()(uphigh9)
    uphigh9 = layers.Activation("relu")(uphigh9)
    uplow9 = layers.BatchNormalization()(uplow9)
    uplow9 = layers.Activation("relu")(uplow9)
    merge9high = concatenate([high1,uphigh9], axis = 3)
    merge9low = concatenate([low1,uplow9], axis = 3)
    high9, low9 = OctaveConv2D(64)([merge9high, merge9low])
    high9 = layers.BatchNormalization()(high9)
    high9 = layers.Activation("relu")(high9)
    low9 = layers.BatchNormalization()(low9)
    low9 = layers.Activation("relu")(low9)
    high9, low9 = OctaveConv2D(64)([high9, low9])
    high9 = layers.BatchNormalization()(high9)
    high9 = layers.Activation("relu")(high9)
    low9 = layers.BatchNormalization()(low9)
    low9 = layers.Activation("relu")(low9)
    conv9 = OctaveConv2D(32, ratio_out=0.0)([high9, low9])
    conv9 = layers.Activation("sigmoid")(conv9)
    conv10 = layers.Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    
    model.summary()
    
    model.compile(optimizer = RAdamOptimizer(learning_rate=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

Overwriting model.py


In [4]:
from model import *
# from data import *

Using TensorFlow backend.


In [0]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)
        
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)
  
def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

In [6]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args, save_to_dir = 'data/membrane/train/aug')
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 128, 128, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
octave_conv2d (OctaveConv2D)    [(None, 256, 256, 56 1152        input_1[0][0]                    
                                                                 average_pooling2d[0][0]          
__________________________________________________________________________________________________
batch_normali

In [7]:
from keras.preprocessing.image import ImageDataGenerator
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.2486 - acc: 0.8913
Epoch 00001: loss improved from inf to 0.24859, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 1287s 644ms/step - loss: 0.2486 - acc: 0.8913
Epoch 2/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.1400 - acc: 0.9420
Epoch 00002: loss improved from 0.24859 to 0.13998, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 1222s 611ms/step - loss: 0.1400 - acc: 0.9420
Epoch 3/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.1096 - acc: 0.9537
Epoch 00003: loss improved from 0.13998 to 0.10964, saving model to unet_membrane.hdf5
2000/2000 [==============================] - 1223s 612ms/step - loss: 0.1096 - acc: 0.9537
Epoch 4/5
1999/2000 [============================>.] - ETA: 0s - loss: 0.0916 - acc: 0.9608
Epoch 00004: loss impr

In [0]:
def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img
        
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

In [9]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 128, 128, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
octave_conv2d_23 (OctaveConv2D) [(None, 256, 256, 56 1152        input_2[0][0]                    
                                                                 average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
batch_normalization_44 (BatchNo (None, 256, 256, 56) 224         octave_conv2d_23[0][0]     